<a href="https://colab.research.google.com/github/anne030303/data-course-sample/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json
import datetime

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 13:26:16--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.3’

All_Beauty.csv.3    100%[===================>]  14.78M  40.9MB/s    in 0.4s    

2022-01-02 13:26:17 (40.9 MB/s) - ‘All_Beauty.csv.3’ saved [15499476/15499476]

--2022-01-02 13:26:17--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.3’

meta_All_Beauty.jso 100%[===================>]   9.85M  32.5MB/s    in 0.3s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
metadata['rank_num'] = pd.to_numeric(metadata['rank'].str.split('in', n = 1, expand = True)[0].str.replace(',',''))
metadata['rank_cat'] = metadata['rank'].str.split('in', n = 1, expand = True)[1].str.replace('(','',regex=True)
metadata['pricenum'] = pd.to_numeric(metadata['price'].str.replace('$','',regex=True), errors='coerce')
metadata = metadata.drop(metadata[(metadata['rank_cat'] != ' Beauty & Personal Care ')].index)
metadata_new = metadata[['asin', 'brand', 'title', 'pricenum', 'rank_num', 'description', 'also_view', 'also_buy']].copy()

In [6]:
metadata_new.head()

,asin,brand,title,pricenum,rank_num,description,also_view,also_buy
0,6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,NaN,2938573.0,[Loud 'N Clear Personal Sound Amplifier allows...,[],[]
1,7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,44.99,872854.0,[No7 Lift & Luminate Triple Action Serum 50ml ...,[],"[B01E7LCSL6, B008X5RVME]"
2,7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,28.76,956696.0,[No7 Stay Perfect Foundation now stays perfect...,"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",[]
3,7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,1870258.0,[],[],[B0041PBXX8]
4,7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,12.15,67701.0,[Lacto Calamine Skin Balance Daily Nourishing ...,"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",[]


## 資料整理

In [7]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [8]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [9]:
ratings_trainings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25


In [10]:
#若為新使用者，則推薦近兩個月的熱銷商品排行(評論數前50中rank前10)
k = 10
ratings_top = ratings_trainings[(ratings_trainings['DATE'] + datetime.timedelta(days = 60)) > '2018-09-01']
ratings_topk = list(ratings_top['asin'].value_counts()[:k].to_dict().keys())
ratings_topk

['B01DKQAXC0',
 'B00W259T7G',
 'B013XKHA4M',
 'B012Z7IHHI',
 'B01AVJCDYA',
 'B0195R1FT8',
 'B01CJNZKZK',
 'B00JVU3K9I',
 'B01DLR9IDI',
 'B018WCT01C']

### 1. Item representation

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_weight = {'title':3, 'brand':2, 'description':1}

# 計算商品用標題所表示的 tfidf 矩陣
#metadata_new['text_data'] = metadata_new.agg(lambda x: f'{" ".join([x["title"]]*text_weight["title"])} {" ".join([x["brand"]]*text_weight["brand"])} {" ".join(x["description"]*text_weight["description"])}', axis=1)
metadata_new['text_data'] = metadata_new.agg(lambda x: f'{" ".join([x["title"]]*text_weight["title"])} {" ".join([x["brand"]]*text_weight["brand"])}', axis=1)
df = metadata_new.drop_duplicates('text_data')
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['text_data'])

metadata_new['text_data'].iloc[0]

"Loud 'N Clear&trade; Personal Sound Amplifier Loud 'N Clear&trade; Personal Sound Amplifier Loud 'N Clear&trade; Personal Sound Amplifier idea village idea village"

### 2. Profiling Learning

In [12]:
# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

In [13]:
# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        items = []
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k+1]
        for index,score in similarity_score:
            if index != item_index:
                items.append([index,score,df['asin'].iloc[index]])
        return items
    except:
        return []


### 3. Recommendation Generation

In [14]:
# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    res = sorted(res, key=lambda x: x[1], reverse=True)
    res = res[:k]
    res = [i[2] for i in res]
    if res == []:
        res = ratings_topk
    return res

## 產生推薦

In [15]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    
    '''
    content-based
    '''
    ratings_trainings = training_data
    recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)


## 結果評估

In [16]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

print('所有使用者推薦評估結果：',evaluate(ratings_testings_by_user, ratings_by_user))

所有使用者推薦評估結果： 0.15084745762711865


#### 單檢視購買過商品的使用者

In [17]:
# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    res = sorted(res, key=lambda x: x[1], reverse=True)
    res = res[:k]
    res = [i[2] for i in res]
    return res

In [18]:
ratings_by_user = recommender(ratings_trainings, users, k = 30)
print('過去未曾評論的新用戶比例：',list(ratings_by_user.values()).count([])/len(ratings_by_user))
ratings_by_olduser = {}
ratings_testings_by_olduser = {}
for key,value in ratings_by_user.items():
    if value != []:
        ratings_by_olduser[key] = value
        ratings_testings_by_olduser[key] = ratings_testings_by_user[key]
        
        if ratings_testings_by_user[key][0] in value:
            print(value,ratings_testings_by_user[key])
print('曾購買商品的使用者推薦評估結果：',evaluate(ratings_testings_by_olduser, ratings_by_olduser))

過去未曾評論的新用戶比例： 0.9417808219178082
曾購買商品的使用者推薦評估結果： 0.0
